## 과제 3: PySpark 커널에서 데이터 탐색 및 쿼리

이 노트북에서는 다음 작업을 수행합니다.

- Amazon SageMaker Studio 노트북을 사용하여 Amazon EMR 클러스터를 시각적으로 찾아보고 이 클러스터로 인증 및 연결하는 방법 알아보기
- PySpark를 사용하여 데이터 탐색 및 쿼리 
- Spark UI를 사용하여 Spark ML 작업을 모니터링

### 과제 3.1: 세션 정보 표시

PySpark 커널을 사용 중이므로 PySpark magic %%info를 사용하여 현재 세션 정보를 표시할 수 있습니다.

In [ ]:
%%info

### 과제 3.2: 데이터 탐색 및 쿼리

PySpark 커널을 사용할 때는 EMR 클러스터에 연결하고 나면 SparkContext 및 HiveContext가 자동으로 생성됩니다. HiveContext를 사용하여 Hive 테이블의 데이터를 쿼리한 다음, Spark 데이터 프레임에서 제공할 수 있습니다.

HiveContext를 사용하여 Hive를 쿼리하고 데이터베이스와 테이블을 관찰합니다.

In [ ]:
#query-hive

sqlContext = HiveContext(sqlContext)

dbs = sqlContext.sql("show databases")
dbs.show()

tables = sqlContext.sql("show tables")
tables.show()

adult 데이터 테이블을 쿼리하여 해당 데이터를 Spark 데이터 프레임으로 가져옵니다.

In [ ]:
#load-data

adult_df = sqlContext.sql("select * from adult_data").cache()

이 데이터 프레임을 사용하여 형태를 살펴보고 데이터 집합의 첫 5개 행을 확인합니다.

In [ ]:
#view-shape

print((adult_df.count(), len(adult_df.columns)))

#Show first 5 rows 
adult_df.head(5)

출력을 더 쉽게 확인할 수 있도록 Spark 데이터 프레임을 Pandas 데이터 프레임으로 변환합니다.

In [ ]:
#convert-dataframe

adult_df.limit(5).toPandas()

선형 회귀와 같은 일부 기계 학습 알고리즘에는 수치형 특성이 필요합니다. 이 실습에서 사용하는 adult 데이터 집합에는 **workclass**, **education**, **occupation**, **marital status**, **relationship**, **race**, **sex** 등의 범주형 특성이 포함되어 있습니다.

다음 코드 블록에는 StringIndexer 및 OneHotEncoderEstimator를 사용하여 범주형 변수를 값이 0과 1인 수치형 변수 집합으로 변환하는 방법이 나와 있습니다.

- StringIndexer는 문자열 값 열을 레이블 인덱스 열로 변환합니다. 
- OneHotEncoderEstimator는 범주형 인덱스 열을 이진 벡터 열에 매핑합니다. 각 행에는 해당 행의 범주형 인덱스를 나타내는 “1”이 최대 1개 포함됩니다.

Spark에서 원-핫 인코딩은 2단계 프로세스로 진행됩니다. 즉, **StringIndexer**와 **OneHotEncoder**를 차례로 사용합니다.

자세한 내용은 [StringIndexer](https://spark.apache.org/docs/latest/ml-features.html#stringindexer) 및 [OneHotEncoder](https://spark.apache.org/docs/latest/ml-features.html#onehotencoder)를 참조하세요.

In [ ]:
#convert-variables

from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator

categorical_variables = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']

indexers = [StringIndexer(inputCol=column, outputCol=column+"-index") for column in categorical_variables]

encoder = OneHotEncoderEstimator(
    inputCols=[indexer.getOutputCol() for indexer in indexers],
    outputCols=["{0}-encoded".format(indexer.getOutputCol()) for indexer in indexers]
)

VectorAssembler 클래스는 여러 열을 입력으로 사용합니다. 그리고 값 배열에 포함된 단일 열을 출력합니다.

이 어셈블러에 관한 자세한 내용은 [VectorAssembler](https://spark.apache.org/docs/latest/ml-features.html#vectorassembler)를 참조하세요.

In [ ]:
#vector-assembler

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=encoder.getOutputCols(),
    outputCol="categorical-features"
)

파이프라인은 변환기와 추정기의 정렬된 목록입니다. 파이프라인을 정의하여 데이터 집합에 변환을 자동으로 반복 적용할 수 있습니다. 이 단계에서는 파이프라인을 정의한 후 데이터 집합에 적용합니다.

**StringIndexer**와 마찬가지로 파이프라인은 **estimator**입니다. 그리고 pipeline.fit() 메서드가 반환하는 **PipelineModel**은 **transformer**입니다.

기계 학습 파이프라인에 관한 자세한 내용은 [파이프라인](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline)을 참조하세요.

In [ ]:
#pyspark-pipelines

from pyspark.ml import Pipeline

# Define the pipeline based on the stages created in previous steps.
pipeline = Pipeline(stages=indexers + [encoder, assembler])

# Define the pipeline model.
pipelineModel = pipeline.fit(adult_df)

# Apply the pipeline model to the dataset.
adult_df = pipelineModel.transform(adult_df)

이전 단계에서 생성된 모든 열을 검토합니다.

In [ ]:
#print-schema

adult_df.printSchema()

변환을 적용하고 나면 인코딩된 모든 범주형 변수가 들어 있는 배열이 열 하나에 포함됩니다.

In [ ]:
#view-categorical-features

adult_df.select('categorical-features').show(truncate=False)

이제 대상 레이블을 인코딩합니다.

In [ ]:
#encode-target

indexer = StringIndexer(inputCol='income', outputCol='label')

adult_df = indexer.fit(adult_df).transform(adult_df)

### 과제 3.3: Spark UI를 사용하여 모니터링 및 디버그

이 섹션에서는 Spark UI를 사용하여 이전 단계에서 실행한 Spark 작업의 성능을 모니터링하고 검사합니다.

현재 Spark 세션 정보를 가져옵니다.

In [ ]:
%%info

**Spark UI** 및 **Driver log**의 하이퍼링크를 확인할 수 있습니다. 이 실습에서 **Driver log** 링크는 비활성화되어 있습니다. 미리 서명된 **Spark UI** URL은 EMR 클러스터 연결 시에 생성됩니다. 이 링크를 선택하면 Spark UI로 이동하여 웹 브라우저에서 Spark 작업 실행을 검사할 수 있습니다. 이러한 지표는 성능 튜닝 시에 유용합니다. 

Spark 서버에서 사용 가능한 몇 가지 중요한 기능은 다음과 같습니다.
- **Jobs** 탭에는 이 Spark 애플리케이션의 모든 Spark 작업 상태가 표시됩니다.
- 요약 섹션 아래의 **Event Timeline** 섹션에는 실행의 다양한 스테이지가 표시됩니다.
- **Completed Jobs** 섹션은 테이블 형식으로 표시됩니다. **Completed Jobs** 섹션 아래에서 작업을 선택하면 해당 작업 내 과제 스테이지에 관한 정보를 검토할 수 있습니다.
- **DAG Visualization**을 사용하면 이전에 실행한 과제를 살펴볼 수 있습니다. **Event Timeline** 보기와 마찬가지로 **DAG Visualization**을 사용하면 스테이지를 선택하여 해당 스테이지 내에서 세부 정보를 확장할 수 있습니다.

### 마무리

축하합니다! 다음을 성공적으로 수행했습니다.

- Amazon SageMaker Studio 노트북을 사용하여 Amazon EMR 클러스터를 시각적으로 찾아보고 이 클러스터로 인증 및 연결하는 방법 알아보기
- PySpark를 사용하여 데이터 탐색 및 쿼리 
- Spark UI를 사용하여 Spark 기계 학습 작업을 모니터링

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 지침으로 돌아갑니다.